# So overall, this is the same as the binary classifier from the first binary classifier, except we also have shorting as well

In this case, we would be executing  trades risking 2% in total of our capital when to buy and when to short. That way we will also make sure not to loose our total capital in general, and we will also simulate liquidation. Meaning, any other stuff that was updated in the first binary classifier we go over and put it in this classifier as well

**Rules:**
-Preprocess the data
-Define model architecture
-Train the model and feed the data to it
-F1-Score & Backtesting

**Live simulation---**
Best to do 50 iterations for all tests in general, look at your python script on your desktop to execute, load, and run them all, after cleaning what needs to be done. Mind you any metatrader5 data we are fetching is being simulated here, nothign is being reflected within our account. 

In [1]:
#making sure we train with our GPU, Pytorch is the fucking shit!!!
import torch
torch.cuda.is_available()

True

In [2]:
torch.cuda.device_count()

1

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.device(0)

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA TITAN RTX'

In [6]:
!pip install MetaTrader5

import os
import shutil
import MetaTrader5 as mt5
import pandas as pd
import datetime
import torch
import torch.nn as nn
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Define the RNN model
class BitcoinPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(BitcoinPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters (ensure these match the training script)
input_size = 10  # Adjusted to match the training script
hidden_size = 2048  # Match with the training script
num_layers = 8  # Match with the training script
output_size = 2  # Binary classification (up or down)

# Load the trained model
model_path = "C:/Users/gunne/Documents/PriceNeuralNetwork/Forex/forexmodel-1.pth"
model = BitcoinPriceLSTM(input_size, hidden_size, num_layers, output_size, dropout=0.5).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("MetaTrader 5 initialized successfully")

# Login to the demo account
account = 51806647  # Ensure the account number is an integer
password = "lKDtOjEK6$rp@f"
server = "ICMarketsSC-Demo"

# Debug print for login details
print(f"Attempting to login with Account: {account}, Password: {password}, Server: {server}")

authorized = mt5.login(login=account, password=password, server=server)
if not authorized:
    print(f"Failed to connect to account #{account}, error code: {mt5.last_error()}")
else:
    print(f"Connected to account #{account}")

MetaTrader 5 initialized successfully
Attempting to login with Account: 51806647, Password: lKDtOjEK6$rp@f, Server: ICMarketsSC-Demo
Connected to account #51806647


In [ ]:
#Trading strategy and logs:
#Now lets do this with the dummy money traded with our account, part 2:
# Trading parameters
import os
import datetime
import time
import pandas as pd
import MetaTrader5 as mt5
from sklearn.preprocessing import StandardScaler
import torch

# Trading parameters
initial_capital = 100
capital = initial_capital
leverage = 200  # 1:200 leverage
profit_cap_percentage = 2.0  # Daily profit target of 2%
stop_loss_percentage = -0.004  # Stop loss at -0.01%
target_capital = 200000
positions = 0
entry_price = None
minimum_percentage = 0.005
cumulative_percentage = 0
profit = 0

# Create directories for logging
base_log_dir = "trading_logs"
os.makedirs(base_log_dir, exist_ok=True)

# Create a new folder for the current test
current_test_index = 0
while os.path.exists(os.path.join(base_log_dir, f"test_{current_test_index}")):
    current_test_index += 1

current_log_dir = os.path.join(base_log_dir, f"test_{current_test_index}")
os.makedirs(current_log_dir)

log_file_path = os.path.join(current_log_dir, "trade_log.txt")
log_csv_path = os.path.join(current_log_dir, "trade_log.csv")
iteration_csv_path = os.path.join(current_log_dir, "iteration_log.csv")

# DataFrames for logs
trade_log_df = pd.DataFrame(columns=['time', 'action', 'price', 'positions', 'capital', 'profit_percentage', 'profit'])
iteration_log_df = pd.DataFrame(columns=['time', 'price', 'positions', 'capital'])

# Function to adjust trade volume
def validate_volume(raw_volume, symbol):
    min_volume = mt5.symbol_info(symbol).volume_min
    max_volume = mt5.symbol_info(symbol).volume_max
    step_volume = mt5.symbol_info(symbol).volume_step

    if raw_volume < min_volume:
        return min_volume
    elif raw_volume > max_volume:
        return max_volume
    else:
        return round(raw_volume / step_volume) * step_volume

# Run the trading loop
while capital < target_capital:
    # Fetch live data
    rates = mt5.copy_rates_from_pos("GBPUSD", mt5.TIMEFRAME_H1, 0, 750)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame.set_index('time', inplace=True)

    # Feature engineering
    rates_frame['Change_Open_Close'] = ((rates_frame['close'] - rates_frame['open']) / rates_frame['open']) * 100
    rates_frame['Change_High_Low'] = ((rates_frame['high'] - rates_frame['low']) / rates_frame['high']) * 100
    rates_frame['Profit_Between_Time_Series'] = rates_frame['Change_Open_Close'].diff().fillna(0)
    rates_frame['Buy_Signal'] = (rates_frame['Profit_Between_Time_Series'] > 0.01).astype(int)
    rates_frame['Sell_Signal'] = rates_frame['Buy_Signal'].shift(-1).fillna(0).astype(int)

    # Prepare features for model prediction
    features = rates_frame.drop(columns=['Buy_Signal', 'Sell_Signal']).values
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Predicting buy signals
    with torch.no_grad():
        inputs = torch.tensor(scaled_features, dtype=torch.float32).to(device)
        outputs = model(inputs.unsqueeze(1))
        predicted_signals = torch.argmax(outputs, dim=1).cpu().numpy()

    rates_frame['Predicted_Buy_Signal'] = predicted_signals

    # Execute trades based on predictions
    current_price = rates_frame['close'].iloc[-1]
    date_time = datetime.datetime.now()

    if capital >= target_capital:
        print(f"Target capital of {target_capital} reached at {date_time}, stopping trading.")
        break

    if rates_frame['Predicted_Buy_Signal'].iloc[-1] == 1 and positions == 0:
        # Enter position
        positions = leverage * (capital / current_price)
        entry_price = current_price
        log_entry = {
            'time': date_time,
            'action': 'enter',
            'price': entry_price,
            'positions': positions,
            'capital': capital,
            'profit_percentage': None,
            'profit': None
        }
        trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_entry])], ignore_index=True)
        with open(log_file_path, "a") as log_file:
            log_file.write(f"Entered position at time {date_time}, price: {entry_price}, positions: {positions}, capital: {capital}\n")

    if positions != 0 and cumulative_percentage != profit_cap_percentage:
        profit_percentage = (current_price - entry_price) / entry_price * 100

        # Determine if the current price > entry price and hits our minimum percentage of change
        if current_price > entry_price and profit_percentage >= minimum_percentage:
            # Exit the positions and add the minimum percentage amount to a cumulative percentage
            # Profit is the entry position 
            profit = (positions * (current_price - entry_price))  # Leverage has already been applied to positions
            capital += profit
            cumulative_percentage += minimum_percentage

            log_exit = {
                'time': date_time,
                'action': 'exit',
                'price': current_price,
                'positions': positions,
                'capital': capital,
                'profit_percentage': profit_percentage,
                'profit': profit
            }
            trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_exit])], ignore_index=True)
            with open(log_file_path, "a") as log_file:
                log_file.write(f"Exited position at time {date_time}, price: {current_price}, profit_percentage: {profit_percentage}, profit: {profit}, capital: {capital}\n")

            # Reset positions
            positions = 0
            entry_price = None

        # Edge case where the profit percentage is greater than or equal to the profit cap percentage, per minute iteration
        if profit_percentage >= profit_cap_percentage:
            # Exit position due to reaching profit cap or stop loss
            profit = (positions * (current_price - entry_price))  # Leverage has already been applied to positions
            capital += profit

            log_exit = {
                'time': date_time,
                'action': 'exit',
                'price': current_price,
                'positions': positions,
                'capital': capital,
                'profit_percentage': profit_percentage,
                'profit': profit
            }
            trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_exit])], ignore_index=True)
            with open(log_file_path, "a") as log_file:
                log_file.write(f"Exited position at time {date_time}, price: {current_price}, profit_percentage: {profit_percentage}, profit: {profit}, capital: {capital}\n")

            # Reset positions
            positions = 0
            entry_price = None

        # In the event we need to stop and liquidate at our stop loss
        if profit_percentage <= stop_loss_percentage:
            # Apply the stop loss percentage and exit the trade
            capital += (capital * stop_loss_percentage)

            log_exit = {
                'time': date_time,
                'action': 'exit',
                'price': current_price,
                'positions': positions,
                'capital': capital,
                'profit_percentage': profit_percentage,
                'profit': profit
            }
            trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_exit])], ignore_index=True)
            with open(log_file_path, "a") as log_file:
                log_file.write(f"Exited position at time {date_time}, price: {current_price}, profit_percentage: {profit_percentage}, profit: {profit}, capital: {capital}\n")

            # Reset positions
            positions = 0
            entry_price = None

    # Print details of each iteration
    iteration_log = {
        'time': date_time,
        'price': current_price,
        'positions': positions,
        'capital': capital
    }
    iteration_log_df = pd.concat([iteration_log_df, pd.DataFrame([iteration_log])], ignore_index=True)
    print(f"Iteration at time {date_time}, current price: {current_price}, positions: {positions}, capital: {capital}")
    with open(log_file_path, "a") as log_file:
        log_file.write(f"Iteration at time {date_time}, current price: {current_price}, positions: {positions}, capital: {capital}\n")

    # Save logs to CSV files
    trade_log_df.to_csv(log_csv_path, index=False)
    iteration_log_df.to_csv(iteration_csv_path, index=False)

    # Wait before the next iteration (e.g., 1 minute)
    time.sleep(60)

    # Reset daily profit at the end of the trading day (assumes a 24-hour trading day for simplicity)
    if date_time.hour == 0 and date_time.minute == 0:
        cumulative_percentage = 0


Iteration at time 2024-05-30 09:41:36.699086, current price: 1.27042, positions: 0, capital: 100


C:\Users\gunne\AppData\Local\Temp\ipykernel_15304\3256594789.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  iteration_log_df = pd.concat([iteration_log_df, pd.DataFrame([iteration_log])], ignore_index=True)


Iteration at time 2024-05-30 09:42:37.039410, current price: 1.27041, positions: 0, capital: 100
Iteration at time 2024-05-30 09:43:37.239626, current price: 1.27045, positions: 0, capital: 100
Iteration at time 2024-05-30 09:44:37.550252, current price: 1.2705199999999999, positions: 0, capital: 100
Iteration at time 2024-05-30 09:45:37.881479, current price: 1.27038, positions: 0, capital: 100
Iteration at time 2024-05-30 09:46:38.200947, current price: 1.27062, positions: 0, capital: 100
Iteration at time 2024-05-30 09:47:38.530989, current price: 1.2707899999999999, positions: 0, capital: 100
Iteration at time 2024-05-30 09:48:38.864943, current price: 1.2708300000000001, positions: 0, capital: 100
Iteration at time 2024-05-30 09:49:39.190458, current price: 1.27089, positions: 0, capital: 100
Iteration at time 2024-05-30 09:50:39.523358, current price: 1.2708599999999999, positions: 0, capital: 100
Iteration at time 2024-05-30 09:51:39.812840, current price: 1.27081, positions: 0,

C:\Users\gunne\AppData\Local\Temp\ipykernel_15304\3256594789.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_entry])], ignore_index=True)


Iteration at time 2024-05-30 10:06:44.144522, current price: 1.27164, positions: 15727.721682237108, capital: 100
Iteration at time 2024-05-30 10:07:44.469025, current price: 1.27164, positions: 15727.721682237108, capital: 100


...

Iteration at time 2024-05-30 13:53:53.442018, current price: 1.27225, positions: 21293.8904518667, capital: 135.45576063693704
Iteration at time 2024-05-30 13:54:53.734078, current price: 1.2724, positions: 0, capital: 138.64984420471433


C:\Users\gunne\AppData\Local\Temp\ipykernel_15304\3256594789.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_entry])], ignore_index=True)


Iteration at time 2024-05-30 13:55:53.935439, current price: 1.27255, positions: 21790.867817329665, capital: 138.64984420471433
Iteration at time 2024-05-30 13:56:54.249204, current price: 1.2727, positions: 0, capital: 141.918474377311


C:\Users\gunne\AppData\Local\Temp\ipykernel_15304\3256594789.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_log_df = pd.concat([trade_log_df, pd.DataFrame([log_entry])], ignore_index=True)


Iteration at time 2024-05-30 13:57:54.561923, current price: 1.27247, positions: 22305.983540250218, capital: 141.918474377311
Iteration at time 2024-05-30 13:58:54.891259, current price: 1.27253, positions: 22305.983540250218, capital: 141.918474377311
Iteration at time 2024-05-30 13:59:55.204270, current price: 1.27243, positions: 22305.983540250218, capital: 141.918474377311
Iteration at time 2024-05-30 14:00:55.531315, current price: 1.27226, positions: 0, capital: 141.35080047980173
Iteration at time 2024-05-30 14:01:55.822918, current price: 1.27241, positions: 0, capital: 141.35080047980173
Iteration at time 2024-05-30 14:02:56.014995, current price: 1.2726600000000001, positions: 0, capital: 141.35080047980173
Iteration at time 2024-05-30 14:03:56.330596, current price: 1.27238, positions: 0, capital: 141.35080047980173
Iteration at time 2024-05-30 14:04:56.621742, current price: 1.27256, positions: 0, capital: 141.35080047980173
Iteration at time 2024-05-30 14:05:56.927048, cu

In [ ]:
#MetaTrader5 implementation
#The model may make predictions but metatrader5 needs to execute based on miliseconds to get ou rprofits